In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../')
import velovae as vv

In [2]:
dataset = 'Erythroid'
root = "/scratch/blaauw_root/blaauw1/gyichen"
adata = anndata.read_h5ad(f'{root}/data/{dataset}_pp.h5ad')
adata.obs["clusters"] = adata.obs["celltype"].to_numpy()

In [ ]:
#vv.preprocess(adata, n_gene=2000, min_shared_counts=20)

In [3]:
model_path_base = f'{root}/checkpoints/{dataset}'
figure_path_base = f'{root}/figures/{dataset}'
data_path = f'{root}/data/velovae/discrete/{dataset}'
gene_plot = ['Smim1', 'Blvrb', 'Hba-x', 'Lmo2']

# Informative Time Prior

In [4]:
capture_time = adata.obs['stage'].to_numpy()
tprior = np.array([float(x[1:]) for x in capture_time])
adata.obs['tprior'] = tprior

# VAE

In [5]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)

dvae = vv.model.VAE(adata, 
                    tmax=20, 
                    dim_z=5, 
                    device='cuda:0',
                    discrete=True,
                    init_method='tprior',
                    init_key='tprior',
                    tprior='tprior',
                    init_ton_zero=False)

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Detected 211 velocity genes.
Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Initialization using prior time.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Initial induction: 126, repression: 874/1000
Using informative time prior.
Learning Rate based on Data Sparsity: 0.0001
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 54, test iteration: 106
*********       Stage 1: Early Stop Triggered at epoch 827.       *********
*********                      Stage  2                       *********
*********             Velocity Refinement Round 1             *********


  0%|          | 0/9815 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.035
Average Set Size: 211
*********     Round 1: Early Stop Triggered at epoch 1242.    *********
*********             Velocity Refinement Round 2             *********
*********     Round 2: Early Stop Triggered at epoch 1329.    *********
Change in x0: 0.0852
*********             Velocity Refinement Round 3             *********
*********     Round 3: Early Stop Triggered at epoch 1567.    *********
Change in x0: 0.0428
*********             Velocity Refinement Round 4             *********
*********     Round 4: Early Stop Triggered at epoch 1605.    *********
Change in x0: 0.0340
*********             Velocity Refinement Round 5             *********
Stage 2: Early Stop Triggered at round 4.
*********              Finished. Total Time =   0 h : 17 m : 52 s             *********
Final: Train ELBO = -1199.218,	Test ELBO = -1220.816


# Full VB

In [6]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)

dvae = vv.model.VAE(adata, 
                    tmax=20, 
                    dim_z=5, 
                    device='cuda:0',
                    discrete=True,
                    full_vb=True,
                    init_method='tprior',
                    init_key='tprior',
                    tprior='tprior',
                    init_ton_zero=False)

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dfullvb', data_path, f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Detected 211 velocity genes.
Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Initialization using prior time.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Initial induction: 126, repression: 874/1000
Using informative time prior.
Learning Rate based on Data Sparsity: 0.0001
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 54, test iteration: 106
*********                      Stage  2                       *********
*********             Velocity Refinement Round 1             *********


  0%|          | 0/9815 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.031
Average Set Size: 197
*********     Round 1: Early Stop Triggered at epoch 1092.    *********
*********             Velocity Refinement Round 2             *********
*********     Round 2: Early Stop Triggered at epoch 1181.    *********
Change in x0: 0.1007
*********             Velocity Refinement Round 3             *********
*********     Round 3: Early Stop Triggered at epoch 1262.    *********
Change in x0: 0.0716
*********             Velocity Refinement Round 4             *********
*********     Round 4: Early Stop Triggered at epoch 1359.    *********
Change in x0: 0.0603
*********             Velocity Refinement Round 5             *********
*********     Round 5: Early Stop Triggered at epoch 1418.    *********
Change in x0: 0.0553
*********             Velocity Refinement Round 6             *********
Stage 2: Early Stop Triggered at round 5.
*********              Finished. Total Time =   0 h : 18 m : 14 s             *********
Final: Tra

# Evaluation

In [7]:
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE', 'Discrete FullVB'],
                 ['dvae', 'dfullvb'],
                 compute_metrics=False,
                 genes=gene_plot,
                 grid_size=(1,4),
                 figure_path=figure_path_base)

---   Plotting  Results   ---
computing velocity graph (using 4/32 cores)


  0%|          | 0/9815 [00:00<?, ?cells/s]

    finished (0:00:33) --> added 
    'dvae_velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:02) --> added
    'dvae_velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file /scratch/blaauw_root/blaauw1/gyichen/figures/Erythroid/eval_dvae_stream.png
computing velocity graph (using 4/32 cores)


  0%|          | 0/9815 [00:00<?, ?cells/s]

    finished (0:00:17) --> added 
    'dfullvb_velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:02) --> added
    'dfullvb_velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file /scratch/blaauw_root/blaauw1/gyichen/figures/Erythroid/eval_dfullvb_stream.png


(None, None)